In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AAC_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "myUserAdmin"
password = "1234"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

df = df.iloc[:,1:]



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    #logo element
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Center(html.Img(
            src = "/files/Grazioso Salvare Logo.png", 
            alt = "logo", 
            width = "300", 
            height = "300"))  
    ], href="https://www.snhu.edu"),
    html.Hr(),
    
    #dropdown list
    html.Div(
        className = "dropdown",
        style = {"width": "360px"},
        children = [
            dcc.Dropdown(
                id = "interactive_filter_option",
                options=[
                    {'label': 'Water Rescue', 'value': 
                        {
                            "breed":{"$regex":"^(Labrador Retr|Chesa Bay Retr|Newfoundland)","$options":"i"},
                            "sex_upon_outcome":"Intact Female",
                            "age_upon_outcome_in_weeks":{ "$gte": 26, "$lt": 156 }
                        }
                    },
                    {'label': 'Mountain or Wilderness Rescue', 'value': 
                        {
                            "breed":{"$regex":"^(German Shepherd|Alaskan Malamute|Old English Sheepdog|Husky|Rottweiler)","$options":"i"},
                            "sex_upon_outcome":"Intact Male",
                            "age_upon_outcome_in_weeks":{ "$gte": 26, "$lt": 156 }
                        }
                    },
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': 
                        {
                            "breed":{"$regex":"^(Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler)","$options":"i"},
                            "sex_upon_outcome":"Intact Male",
                            "age_upon_outcome_in_weeks":{ "$gte": 20, "$lt": 300 }
                        }},
                    {'label': 'View All', 'value': {}},
                ],
                value={},
                placeholder = "Filter Database For..."
            )
        ]
    ),
    html.Hr(),

    
    #data table element
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "milti",
        column_selectable = True,
        row_selectable = True,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        style_header={'paddingRight':'25px'}

    ),
    
    #slider to set page size
    html.Div(
        children = [
            html.Div(
                children = "results per page: 4",
                id='slider_output',
                style={'marginTop' : "6px",'width' : '200px', 'marginRight' : '20px', 'marginLeft' : 'auto', 'display': 'inlineBlock'}
            ),
            html.Div(
                dcc.Slider(
                    min = 0, 
                    max = 4, 
                    step = 0.1,
                    marks={i: '{}'.format(10 ** i) for i in range(5)},
                    value=1,
                    id='page-size-slider'
                ),
                style={'width' : '300px','marginRight' : '20px', 'marginLeft' : '0px', 'display': 'inlineBlock'}
            )
        ],
        style={'display': 'flex', 'marginTop' : "5px"}
    ),
    html.Div(html.Hr(),style={'marginTop' : "-8px"}),
    
    #charts
    html.Div(
        children = [
            #pie chart
            html.Div(
                children = dcc.Graph(id="pie-chart"),
                style={'width': '49%', 'height': '500px','display': 'inlineBlock'}
            ),
            #geolocation element
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '49%','display': 'inlineBlock'}
            )
        ],
        style = {"display":"flex"}
    ),
    html.Hr(),
    
    #ME!
    html.Center(html.B(html.H3('Interface by: Marc McClure'))),
])

#############################################
# Interaction Between Components / Controller
#############################################

#callback for filtering datatable from dropdown list
@app.callback(
    Output(component_id='datatable-id', component_property = 'data'),
    [
        Input(component_id='interactive_filter_option',component_property = 'value')
    ]
)
def on_dropdown_selection(ddVal):
    query = ddVal
    df = pd.DataFrame(list(shelter.read(query)))
    df = df.iloc[:,1:]
    return df.to_dict('records')

#callback for setting results per page with slider
@app.callback(
    [
        Output('slider_output', 'children'),
        Output('datatable-id', 'page_size')
    ],
    [
        Input('page-size-slider', 'value')
    ]
)
def display_and_set_value(value):
    resultsPerPage = int(10**value)
    return "Results per Page: " + str(resultsPerPage), resultsPerPage

#This callback will highlight a colum on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [
        Input('datatable-id', 'selected_columns')
    ]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#pie chart callback
@app.callback(
    Output('pie-chart', "figure"),
    [
        Input('datatable-id', "derived_viewport_data"),
        Input('datatable-id', "columns"),
        Input('datatable-id', 'selected_columns')
    ]
)
def update_pie_chart(viewData,columns,selected_columns):
    dff = pd.DataFrame.from_dict(viewData)
    
    if len(selected_columns) == 0:
        values_in_selected_colums = dff["animal_type"].values
    elif len(selected_columns) == 1:
        values_in_selected_colums = dff[selected_columns[0]].values
    else:   
        values_in_selected_colums = np.transpose([dff[i].values for i in selected_columns])
        values_in_selected_colums = np.array(["~".join(i) for i in values_in_selected_colums])
    unique_values = np.unique(values_in_selected_colums)
    count_of_values = [np.count_nonzero(values_in_selected_colums == i) for i in unique_values]
    
    my_chart = px.pie(
        names = unique_values, 
        values = count_of_values
    )
        
    return my_chart

# Map Callback
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_viewport_data"),
        Input('datatable-id', 'selected_rows')
    ]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    markers = [dl.Marker(position=[dff.iloc[i,13],dff.iloc[i,14]], children=[
                dl.Tooltip(dff.iloc[i,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[i,9])
                ])
            ]) for i in selected_rows]
    if len(selected_rows) == 0:
        my_center = [30.75,-97.48]
        my_zoom = 8
    elif len(selected_rows) == 1:
        my_center = [dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]]
        my_zoom = 12
    else :
        lat_list = [dff.iloc[i,13] for i in selected_rows]
        lon_list = [dff.iloc[i,14] for i in selected_rows]
        my_center = [(max(lat_list)+min(lat_list))/2,(max(lon_list)+min(lon_list))/2]
        my_zoom = -max([
            np.log2((max(lat_list)-min(lat_list))/180), 
            np.log2((max(lon_list)-min(lon_list))/360),
            -12])
        
    my_map =  [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=my_center, zoom=my_zoom, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.LayerGroup(markers)
        ])
    ]
    return my_map

app